# 🎤 CLARISSA Voice Input Showcase

**Talk to Your Reservoir Simulation**

This notebook demonstrates CLARISSA's voice interface - control reservoir simulations through natural speech.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/wolfram-laube/clarissa/blob/main/docs/tutorials/notebooks/16_Voice_Input_Showcase.ipynb)

---

## What You'll Learn

1. **Speech-to-Text** - Convert voice to text with Whisper
2. **Intent Recognition** - Parse commands into structured intents
3. **Command Execution** - Trigger visualizations by voice
4. **Full Pipeline** - End-to-end voice control demo

## 1️⃣ Setup & Installation

In [ ]:
# Install required packages
!pip install -q openai anthropic plotly numpy ipywidgets

# For local Whisper (optional - skip if using API)
# !pip install -q faster-whisper

print("✅ Packages installed (OpenAI + Anthropic)")

In [ ]:
import os
import json
import base64
from dataclasses import dataclass, field
from typing import Dict, Any, Optional, List
from enum import Enum
import numpy as np
import plotly.graph_objects as go
from IPython.display import display, HTML, Audio
import ipywidgets as widgets

print("✅ Imports ready")

In [ ]:
# API Key Setup - Choose your LLM provider
# ═══════════════════════════════════════════════════════════════
# CLARISSA supports both OpenAI and Anthropic (Claude) for intent parsing.
# Whisper (OpenAI) is always used for speech-to-text.
# ═══════════════════════════════════════════════════════════════

# Option 1: Colab secrets (recommended)
try:
    from google.colab import userdata

    # Try Anthropic first (CLARISSA's native LLM)
    try:
        os.environ['ANTHROPIC_API_KEY'] = userdata.get('ANTHROPIC_API_KEY')
        print("✅ Anthropic API key loaded (Claude)")
    except:
        pass

    # Also try OpenAI (for Whisper STT + optional intent parsing)
    try:
        os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
        print("✅ OpenAI API key loaded (Whisper STT)")
    except:
        pass

except ImportError:
    pass

# Check what's available
anthropic_key = os.getenv('ANTHROPIC_API_KEY')
openai_key = os.getenv('OPENAI_API_KEY')

print()
print("═" * 50)
print("LLM Configuration:")
print("═" * 50)

if anthropic_key:
    print("🟢 Claude (Anthropic): Available - will use for intent parsing")
    LLM_PROVIDER = "anthropic"
elif openai_key:
    print("🟢 GPT-4 (OpenAI): Available - will use for intent parsing")
    LLM_PROVIDER = "openai"
else:
    print("🟡 No LLM API key - using rule-based parsing only")
    print("   (Works for common commands, limited for complex queries)")
    LLM_PROVIDER = "rules"

if openai_key:
    print("🟢 Whisper STT: Available")
else:
    print("🟡 Whisper STT: Not available (no OpenAI key)")
    print("   (Text input mode only)")

print("═" * 50)
print()
print("💡 To add API keys in Colab:")
print("   1. Click 🔑 Secrets (left sidebar)")
print("   2. Add: ANTHROPIC_API_KEY and/or OPENAI_API_KEY")
print("   3. Enable notebook access")
print("   4. Restart runtime")

## 2️⃣ Core Voice Components

These are simplified versions of the full CLARISSA voice module.

In [ ]:
# Intent Types
class IntentType(Enum):
    VISUALIZE_PROPERTY = "visualize_property"
    QUERY_VALUE = "query_value"
    NAVIGATE = "navigate"
    HELP = "help"
    CANCEL = "cancel"
    CONFIRM = "confirm"
    UNKNOWN = "unknown"

@dataclass
class Intent:
    """Parsed intent from voice command."""
    type: IntentType
    confidence: float
    slots: Dict[str, Any] = field(default_factory=dict)
    raw_text: str = ""

@dataclass
class VoiceResponse:
    """Response to voice command."""
    success: bool
    text: str
    intent: Optional[Intent] = None
    visualization: Optional[go.Figure] = None

print("✅ Data classes defined")

In [ ]:
# Domain vocabulary for better recognition
DOMAIN_VOCABULARY = """
Reservoir simulation terms: permeability, porosity, water saturation,
oil saturation, pressure, BHP, bottomhole pressure, OOIP,
waterflood, injector, producer, PROD1, INJ1, INJ2, INJ3, INJ4,
millidarcy, mD, psi, bbl/day, STB, FOPT, FOPR, FWPT, FWPR, FWCT,
water cut, layer, grid, cell, timestep, 3D, cross-section, animation
"""

print("📚 Domain vocabulary loaded")

### 2.1 Speech-to-Text (Whisper)

In [ ]:
def transcribe_audio(audio_path: str) -> str:
    """
    Transcribe audio file using OpenAI Whisper API.

    Args:
        audio_path: Path to audio file (WAV, MP3, etc.)

    Returns:
        Transcribed text
    """
    import openai

    client = openai.OpenAI()

    with open(audio_path, 'rb') as audio_file:
        response = client.audio.transcriptions.create(
            model="whisper-1",
            file=audio_file,
            prompt=DOMAIN_VOCABULARY,
            language="en"
        )

    return response.text

print("🎤 Whisper transcription function ready")

### 2.2 Intent Parser (LLM-based)

In [ ]:
INTENT_PROMPT = """You are a reservoir simulation assistant. Parse the user's voice command into a structured intent.

Available intents:
- visualize_property: Show reservoir properties (permeability, porosity, saturation, pressure)
- query_value: Ask about simulation values (rates, pressures, water cut, cumulative production)
- navigate: Go to different sections (results, sensitivity, model)
- help: Ask for help or guidance
- cancel: Stop or cancel current action
- confirm: Confirm a pending action

Slots to extract:
- property: permeability, porosity, water_saturation, oil_saturation, pressure
- layer: integer (1-5 typically)
- time_days: integer (simulation day)
- view_type: 3d, cross_section_xy, cross_section_xz, animation
- well: PROD1, INJ1, INJ2, INJ3, INJ4
- target: results, sensitivity, model, export

User said: "{text}"

Respond with ONLY valid JSON (no markdown, no explanation):
{{"intent": "<type>", "confidence": <0.0-1.0>, "slots": {{...}}}}"""

import re

def parse_intent_rules(text: str) -> Intent:
    """Rule-based intent parsing - works WITHOUT API key."""
    text_lower = text.lower().strip()
    slots = {}

    # Cancel / Confirm / Help
    if text_lower in ["stop", "cancel", "never mind", "abort", "quit"]:
        return Intent(IntentType.CANCEL, 1.0, {}, text)
    if text_lower in ["yes", "yeah", "yep", "confirm", "ok", "okay", "do it", "go ahead"]:
        return Intent(IntentType.CONFIRM, 1.0, {}, text)
    if text_lower == "help" or "what can" in text_lower:
        return Intent(IntentType.HELP, 1.0, {}, text)

    # Visualization
    viz_keywords = ["show", "display", "visualize", "plot", "view", "see"]
    if any(kw in text_lower for kw in viz_keywords):
        if "perm" in text_lower: slots["property"] = "permeability"
        elif "poro" in text_lower: slots["property"] = "porosity"
        elif "saturation" in text_lower or " sw" in text_lower: slots["property"] = "water_saturation"
        elif "pressure" in text_lower: slots["property"] = "pressure"

        layer_match = re.search(r'layer\s*(\d+)', text_lower)
        if layer_match: slots["layer"] = int(layer_match.group(1))

        time_match = re.search(r'(?:day|time)\s*(\d+)', text_lower)
        if time_match: slots["time_days"] = int(time_match.group(1))

        if not slots.get("property") and not slots.get("layer"):
            slots["property"] = "permeability"

        return Intent(IntentType.VISUALIZE_PROPERTY, 0.95, slots, text)

    # Query
    if any(kw in text_lower for kw in ["what", "how much", "tell me"]):
        if "oil rate" in text_lower: slots["property"] = "oil_rate"
        elif "water cut" in text_lower: slots["property"] = "water_cut"
        elif "water rate" in text_lower: slots["property"] = "water_rate"
        elif "pressure" in text_lower: slots["property"] = "pressure"
        if slots:
            return Intent(IntentType.QUERY_VALUE, 0.9, slots, text)

    return None


def parse_with_claude(text: str) -> Intent:
    """Parse intent using Claude (Anthropic)."""
    import anthropic
    client = anthropic.Anthropic()

    response = client.messages.create(
        model="claude-sonnet-4-20250514",
        max_tokens=200,
        messages=[{"role": "user", "content": INTENT_PROMPT.format(text=text)}]
    )

    result_text = response.content[0].text.strip()
    if result_text.startswith("```"):
        result_text = result_text.split("```")[1].replace("json", "", 1)

    data = json.loads(result_text)
    return Intent(
        IntentType(data.get("intent", "unknown")),
        float(data.get("confidence", 0.8)),
        data.get("slots", {}),
        text
    )


def parse_with_openai(text: str) -> Intent:
    """Parse intent using GPT-4 (OpenAI)."""
    import openai
    client = openai.OpenAI()

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "Parse voice commands into JSON. Respond with ONLY valid JSON."},
            {"role": "user", "content": INTENT_PROMPT.format(text=text)}
        ],
        temperature=0.1,
        max_tokens=200
    )

    result_text = response.choices[0].message.content.strip()
    if result_text.startswith("```"):
        result_text = result_text.split("```")[1].replace("json", "", 1)

    data = json.loads(result_text)
    return Intent(
        IntentType(data.get("intent", "unknown")),
        float(data.get("confidence", 0.8)),
        data.get("slots", {}),
        text
    )


def parse_intent(text: str) -> Intent:
    """
    Parse text into structured intent.

    Priority:
    1. Rule-based (instant, no API)
    2. Claude (CLARISSA native)
    3. OpenAI GPT-4
    4. Fallback to unknown
    """
    # Try rules first
    rule_result = parse_intent_rules(text)
    if rule_result is not None:
        print(f"   📋 Parsed with rules")
        return rule_result

    # Try Claude (CLARISSA's native LLM)
    if os.getenv('ANTHROPIC_API_KEY'):
        try:
            result = parse_with_claude(text)
            print(f"   🤖 Parsed with Claude")
            return result
        except Exception as e:
            print(f"   ⚠️ Claude error: {e}")

    # Try OpenAI
    if os.getenv('OPENAI_API_KEY'):
        try:
            result = parse_with_openai(text)
            print(f"   🤖 Parsed with GPT-4")
            return result
        except Exception as e:
            print(f"   ⚠️ OpenAI error: {e}")

    # No LLM available
    print(f"   ℹ️ No LLM available for complex command")
    return Intent(IntentType.UNKNOWN, 0.0, {}, text)

print("🧠 Intent parser ready")
print("   Priority: Rules → Claude → GPT-4 → Fallback")

### 2.3 Visualization Generator

In [ ]:
# Create synthetic reservoir data for demo
NX, NY, NZ = 10, 10, 5

def generate_demo_data():
    """Generate synthetic reservoir properties."""
    np.random.seed(42)

    # Permeability with channel
    perm = np.random.lognormal(mean=4.5, sigma=0.5, size=(NX, NY, NZ))
    perm[3:7, :, :] *= 3  # High-perm channel

    # Porosity correlated with perm
    poro = 0.15 + 0.1 * (np.log(perm) - 4) / 2
    poro = np.clip(poro, 0.05, 0.35)

    # Water saturation (varies with time)
    def get_saturation(time_days):
        progress = min(time_days / 1800, 1.0)
        sw = np.ones((NX, NY, NZ)) * 0.2  # Connate water
        # Water front moving from injectors
        for i in range(NX):
            for j in range(NY):
                dist = np.sqrt((i - NX//2)**2 + (j - NY//2)**2)
                if dist < progress * NX * 0.7:
                    sw[i, j, :] = 0.2 + 0.5 * (1 - dist / (NX * 0.7))
        return np.clip(sw, 0.2, 0.8)

    return {
        'permeability': perm,
        'porosity': poro,
        'get_saturation': get_saturation
    }

DEMO_DATA = generate_demo_data()
print("📊 Demo data generated")
print(f"   Grid: {NX}×{NY}×{NZ} = {NX*NY*NZ} cells")

In [ ]:
def create_3d_visualization(prop_name: str, prop_data: np.ndarray) -> go.Figure:
    """Create 3D scatter plot of property."""
    x, y, z = [], [], []
    values = []

    for i in range(NX):
        for j in range(NY):
            for k in range(NZ):
                x.append(i)
                y.append(j)
                z.append(k)
                values.append(prop_data[i, j, k])

    fig = go.Figure(data=go.Scatter3d(
        x=x, y=y, z=z,
        mode='markers',
        marker=dict(
            size=8,
            color=values,
            colorscale='Viridis',
            colorbar=dict(title=prop_name.title()),
            opacity=0.8
        )
    ))

    fig.update_layout(
        title=f"3D {prop_name.title()} Distribution",
        scene=dict(
            xaxis_title="X",
            yaxis_title="Y",
            zaxis_title="Layer",
            aspectmode='cube'
        ),
        height=500
    )

    return fig

def create_cross_section(prop_name: str, prop_data: np.ndarray, layer: int) -> go.Figure:
    """Create 2D heatmap cross-section."""
    layer_idx = max(0, min(layer - 1, NZ - 1))
    data_2d = prop_data[:, :, layer_idx]

    fig = go.Figure(data=go.Heatmap(
        z=data_2d.T,
        colorscale='Viridis',
        colorbar=dict(title=prop_name.title())
    ))

    # Add well markers
    fig.add_trace(go.Scatter(
        x=[NX//2], y=[NY//2],
        mode='markers+text',
        marker=dict(size=15, color='blue', symbol='circle'),
        text=['PROD1'], textposition='top center',
        name='Producer'
    ))

    # Injectors at corners
    inj_x = [1, 1, NX-2, NX-2]
    inj_y = [1, NY-2, 1, NY-2]
    inj_names = ['INJ1', 'INJ2', 'INJ3', 'INJ4']
    fig.add_trace(go.Scatter(
        x=inj_x, y=inj_y,
        mode='markers+text',
        marker=dict(size=12, color='red', symbol='triangle-up'),
        text=inj_names, textposition='top center',
        name='Injectors'
    ))

    fig.update_layout(
        title=f"{prop_name.title()} at Layer {layer}",
        xaxis_title="X",
        yaxis_title="Y",
        height=450
    )

    return fig

print("🎨 Visualization functions ready")

### 2.4 Command Executor

In [ ]:
def execute_intent(intent: Intent) -> VoiceResponse:
    """
    Execute parsed intent and return response.

    Args:
        intent: Parsed Intent object

    Returns:
        VoiceResponse with text and optional visualization
    """
    slots = intent.slots

    if intent.type == IntentType.VISUALIZE_PROPERTY:
        prop = slots.get('property', 'permeability')
        layer = slots.get('layer')
        time_days = slots.get('time_days', 500)
        view_type = slots.get('view_type', '3d')

        # Get property data
        if 'saturation' in prop or prop == 'sw':
            prop_data = DEMO_DATA['get_saturation'](time_days)
            prop_name = 'water_saturation'
        elif prop in ['permeability', 'perm']:
            prop_data = DEMO_DATA['permeability']
            prop_name = 'permeability'
        elif prop in ['porosity', 'poro']:
            prop_data = DEMO_DATA['porosity']
            prop_name = 'porosity'
        else:
            prop_data = DEMO_DATA['permeability']
            prop_name = 'permeability'

        # Create visualization
        if layer or 'cross' in str(view_type):
            layer = layer or 3
            fig = create_cross_section(prop_name, prop_data, layer)
            text = f"Showing {prop_name.replace('_', ' ')} at layer {layer}."
        else:
            fig = create_3d_visualization(prop_name, prop_data)
            text = f"Showing {prop_name.replace('_', ' ')} in 3D."

        return VoiceResponse(True, text, intent, fig)

    elif intent.type == IntentType.QUERY_VALUE:
        prop = slots.get('property', 'oil_rate')

        # Simulate query results
        values = {
            'oil_rate': ('1,250', 'bbl/day'),
            'water_rate': ('450', 'bbl/day'),
            'water_cut': ('26', '%'),
            'pressure': ('3,450', 'psi'),
            'bhp': ('3,450', 'psi'),
            'cumulative_oil': ('2.3', 'MMSTB'),
            'fopt': ('2.3', 'MMSTB'),
        }

        prop_key = prop.lower().replace(' ', '_')
        if prop_key in values:
            val, unit = values[prop_key]
            text = f"The {prop.replace('_', ' ')} is {val} {unit}."
        else:
            text = f"I don't have data for {prop}."

        return VoiceResponse(True, text, intent)

    elif intent.type == IntentType.HELP:
        text = """You can say things like:
• "Show me the permeability"
• "Show layer 3"
• "What's the oil rate?"
• "Show saturation at day 500"
• "Stop" or "Cancel" to abort"""
        return VoiceResponse(True, text, intent)

    elif intent.type == IntentType.CANCEL:
        return VoiceResponse(True, "Cancelled.", intent)

    elif intent.type == IntentType.CONFIRM:
        return VoiceResponse(True, "Confirmed.", intent)

    else:
        return VoiceResponse(
            False,
            "I didn't understand that. Try saying 'help' for available commands.",
            intent
        )

print("⚡ Command executor ready")

## 3️⃣ Voice Pipeline

The complete voice processing pipeline:

In [ ]:
def process_voice_command(text_or_audio: str, is_audio: bool = False) -> VoiceResponse:
    """
    Complete voice command processing pipeline.

    Args:
        text_or_audio: Either text command or path to audio file
        is_audio: True if input is audio file path

    Returns:
        VoiceResponse with result
    """
    print("\n" + "="*50)

    # Step 1: Transcribe if audio
    if is_audio:
        print("🎤 Step 1: Transcribing audio...")
        try:
            text = transcribe_audio(text_or_audio)
            print(f"   Transcription: \"{text}\"")
        except Exception as e:
            return VoiceResponse(False, f"Transcription failed: {e}")
    else:
        text = text_or_audio
        print(f"📝 Input: \"{text}\"")

    # Step 2: Parse intent
    print("\n🧠 Step 2: Parsing intent...")
    intent = parse_intent(text)
    print(f"   Intent: {intent.type.value}")
    print(f"   Confidence: {intent.confidence:.0%}")
    if intent.slots:
        print(f"   Slots: {intent.slots}")

    # Step 3: Execute
    print("\n⚡ Step 3: Executing command...")
    response = execute_intent(intent)

    # Step 4: Response
    print(f"\n💬 Response: {response.text}")
    print("="*50)

    return response

print("🚀 Voice pipeline ready!")

## 4️⃣ Interactive Demo

Try voice commands! (Text mode - type what you would say)

In [ ]:
# Demo 1: Show permeability in 3D
response = process_voice_command("show me the permeability")
if response.visualization:
    response.visualization.show()

In [ ]:
# Demo 2: Cross-section at specific layer
response = process_voice_command("show layer 3")
if response.visualization:
    response.visualization.show()

In [ ]:
# Demo 3: Query a value
response = process_voice_command("what is the water cut?")

In [ ]:
# Demo 4: Show saturation at specific time
response = process_voice_command("show water saturation at day 1000")
if response.visualization:
    response.visualization.show()

In [ ]:
# Demo 5: Help command
response = process_voice_command("help")

## 5️⃣ Interactive Widget

Try your own commands:

In [ ]:
# Create interactive widget
text_input = widgets.Text(
    placeholder='Type a command (e.g., "show porosity in 3D")',
    description='🎤 Say:',
    layout=widgets.Layout(width='80%')
)

output = widgets.Output()

def on_submit(change):
    with output:
        output.clear_output()
        if text_input.value:
            response = process_voice_command(text_input.value)
            if response.visualization:
                display(response.visualization)

text_input.on_submit(lambda x: on_submit(x))

submit_btn = widgets.Button(description="Process", button_style='primary')
submit_btn.on_click(lambda x: on_submit(x))

display(widgets.HBox([text_input, submit_btn]))
display(output)

print("\n💡 Type a command and press Enter or click Process")
print("\nExample commands:")
print('  • "show me porosity"')
print('  • "show saturation at layer 2"')
print('  • "what is the oil rate"')
print('  • "help"')

## 6️⃣ Audio File Demo

Upload an audio file to test real speech-to-text:

In [ ]:
# File upload widget
from google.colab import files

def process_uploaded_audio():
    """Upload and process an audio file."""
    print("📁 Upload an audio file (WAV, MP3, M4A)...")
    uploaded = files.upload()

    for filename in uploaded.keys():
        print(f"\n🎵 Processing: {filename}")
        response = process_voice_command(filename, is_audio=True)

        if response.visualization:
            display(response.visualization)

        return response

# Uncomment to test with audio upload:
# process_uploaded_audio()

## 6️⃣ Interactive Voice Input 🎤

### ✅ Voice Recording Works in Colab!

Using `google.colab.output.eval_js()` we can access your microphone directly from Colab. 

**How it works:**
1. Run the cell below
2. Click **"Start Recording"**
3. **Allow microphone access** when your browser asks
4. Speak your command (e.g., *"show me the permeability"*)
5. Click **"Stop & Process"**

**Example commands to try:**
- *"Show me the permeability"*
- *"What is the water cut"*
- *"Display pressure at layer 5"*
- *"Help"*

> 💡 **Tip:** For Whisper transcription, set `OPENAI_API_KEY`. Without it, the system uses a demo transcript but still tests the intent parser.

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════
# 🎤 VOICE RECORDING - Works in Colab!
# ═══════════════════════════════════════════════════════════════════════════

from IPython.display import display, Javascript, HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import os

RECORD_JS = """
async function recordAudio() {
    const div = document.createElement('div');
    div.style.cssText = 'font-family: -apple-system, sans-serif; padding: 25px; background: linear-gradient(135deg, #1a1a2e, #16213e); border-radius: 16px; text-align: center; color: white; max-width: 420px; margin: 10px auto; box-shadow: 0 8px 32px rgba(0,0,0,0.3);';
    
    const title = document.createElement('h3');
    title.textContent = '🎤 CLARISSA Voice Recording';
    title.style.cssText = 'margin: 0 0 5px 0; font-size: 1.3em;';
    
    const subtitle = document.createElement('p');
    subtitle.textContent = 'Speak to control reservoir simulation';
    subtitle.style.cssText = 'color: #888; margin: 0 0 20px 0; font-size: 0.9em;';
    
    const status = document.createElement('p');
    status.textContent = '👆 Click Start to begin';
    status.style.cssText = 'color: #888; margin: 15px 0; font-size: 1.1em; min-height: 30px;';
    status.id = 'rec-status';
    
    const startBtn = document.createElement('button');
    startBtn.textContent = '🎤 Start Recording';
    startBtn.style.cssText = 'padding: 15px 30px; font-size: 16px; font-weight: 600; background: linear-gradient(145deg, #e94560, #c23a51); color: white; border: none; border-radius: 50px; cursor: pointer; margin: 5px; box-shadow: 0 4px 15px rgba(233,69,96,0.4); transition: transform 0.2s;';
    
    const stopBtn = document.createElement('button');
    stopBtn.textContent = '⏹️ Stop & Process';
    stopBtn.style.cssText = 'padding: 15px 30px; font-size: 16px; font-weight: 600; background: linear-gradient(145deg, #4caf50, #388e3c); color: white; border: none; border-radius: 50px; cursor: pointer; margin: 5px; box-shadow: 0 4px 15px rgba(76,175,80,0.4); display: none;';
    
    const hint = document.createElement('p');
    hint.innerHTML = '💡 <em>Try: "show permeability" or "what is water cut"</em>';
    hint.style.cssText = 'color: #666; font-size: 0.85em; margin-top: 20px;';
    
    div.appendChild(title);
    div.appendChild(subtitle);
    div.appendChild(startBtn);
    div.appendChild(stopBtn);
    div.appendChild(status);
    div.appendChild(hint);
    document.body.appendChild(div);
    
    await new Promise(resolve => startBtn.onclick = resolve);
    
    try {
        status.textContent = '🔄 Requesting microphone access...';
        status.style.color = '#00d9ff';
        
        const stream = await navigator.mediaDevices.getUserMedia({audio: true});
        const recorder = new MediaRecorder(stream);
        const chunks = [];
        
        recorder.ondataavailable = e => chunks.push(e.data);
        
        startBtn.style.display = 'none';
        stopBtn.style.display = 'inline-block';
        status.innerHTML = '🔴 <strong>Recording...</strong> Speak now!';
        status.style.color = '#ff6b6b';
        
        recorder.start();
        
        await new Promise(resolve => stopBtn.onclick = resolve);
        
        status.textContent = '⏳ Processing audio...';
        status.style.color = '#00d9ff';
        stopBtn.disabled = true;
        
        recorder.stop();
        
        const blob = await new Promise(resolve => {
            recorder.onstop = () => resolve(new Blob(chunks, {type: 'audio/webm'}));
        });
        
        const reader = new FileReader();
        const base64 = await new Promise(resolve => {
            reader.onloadend = () => resolve(reader.result);
            reader.readAsDataURL(blob);
        });
        
        stream.getAudioTracks()[0].stop();
        status.textContent = '✅ Recording complete!';
        status.style.color = '#4caf50';
        
        return base64;
        
    } catch(err) {
        status.innerHTML = '❌ <strong>Error:</strong> ' + err.message;
        status.style.color = '#ff6b6b';
        hint.innerHTML = '💡 Make sure to <strong>allow microphone access</strong> when prompted';
        return 'ERROR:' + err.message;
    }
}
"""

def record_and_process():
    """Record voice and process with CLARISSA."""
    print("═" * 60)
    print("🎤 CLARISSA Voice Input")
    print("═" * 60)
    print()
    
    # Display the recording UI
    display(Javascript(RECORD_JS))
    
    # Wait for recording (this blocks until user clicks stop)
    print("⏳ Waiting for recording...")
    result = eval_js('recordAudio()')
    
    if not result or result.startswith('ERROR:'):
        print(f"❌ Recording failed: {result}")
        return
    
    # Decode audio
    audio_data = result.split(',')[1]
    audio_bytes = b64decode(audio_data)
    print(f"✅ Recorded {len(audio_bytes):,} bytes of audio")
    
    # Save for playback
    with open('/tmp/recording.webm', 'wb') as f:
        f.write(audio_bytes)
    
    # Playback option
    print()
    display(Audio('/tmp/recording.webm'))
    
    print()
    print("─" * 60)
    print("📝 TRANSCRIPTION & INTENT PARSING")
    print("─" * 60)
    
    # Transcribe with Whisper
    transcript = None
    if os.getenv("OPENAI_API_KEY"):
        try:
            from openai import OpenAI
            client = OpenAI()
            with open('/tmp/recording.webm', 'rb') as f:
                result = client.audio.transcriptions.create(
                    model="whisper-1", file=f, language="en"
                )
            transcript = result.text
            print(f"🗣️ You said: \"{transcript}\"")
        except Exception as e:
            print(f"⚠️ Whisper error: {e}")
    
    if not transcript:
        print("⚠️ No OPENAI_API_KEY - using demo transcript")
        transcript = "show me the permeability"
        print(f"🗣️ Demo: \"{transcript}\"")
    
    # Parse intent
    print()
    intent = parse_intent(transcript)
    print(f"🎯 Intent: {intent.type.value}")
    print(f"📊 Confidence: {intent.confidence:.0%}")
    if intent.slots:
        print(f"📦 Slots: {intent.slots}")
    
    # Generate response
    if intent.type == IntentType.VISUALIZE_PROPERTY:
        p = intent.slots.get('property', 'data')
        l = intent.slots.get('layer')
        t = intent.slots.get('time_days')
        response = f"Visualizing {p}" + (f" at layer {l}" if l else "") + (f" at day {t}" if t else "") + "."
    elif intent.type == IntentType.QUERY_VALUE:
        response = f"Querying {intent.slots.get('property', 'value')}..."
    else:
        response = {
            IntentType.CANCEL: "Operation cancelled.",
            IntentType.CONFIRM: "Confirmed! Executing...",
            IntentType.HELP: "Available: show [property], what is [value], help, cancel",
            IntentType.UNKNOWN: "I didn't understand. Try 'show permeability'.",
        }.get(intent.type, "Processing...")
    
    print()
    print(f"💬 CLARISSA: {response}")
    print()
    print("═" * 60)

# Run it!
record_and_process()


## 8️⃣ SPE Benchmark Test Suite 🧪

Tests based on **SPE Comparative Solution Projects** - industry-standard reservoir simulation benchmarks:

| Model | Grid | Description |
|-------|------|-------------|
| **SPE1** | 10×10×3 | Simple black-oil, gas injection |
| **SPE9** | 24×25×15 | Waterflood, 26 wells |
| **SPE10** | 60×220×85 | Upscaling benchmark (1.1M cells) |

Run the cell below to test intent parsing with **real-world reservoir engineering queries**!

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════
# 🧪 SPE BENCHMARK TEST SUITE
# ═══════════════════════════════════════════════════════════════════════════
# Tests based on SPE Comparative Solution Projects (industry-standard benchmarks)

from dataclasses import dataclass
from typing import List, Optional, Callable, Dict, Any

@dataclass  
class TestResult:
    name: str
    passed: bool
    error: Optional[str] = None

class SPETestRunner:
    """Test runner with SPE model context."""
    
    def __init__(self):
        self.results: List[TestResult] = []
        self._tests: List[Callable] = []
    
    def test(self, name: str, model: str = ""):
        """Decorator to register a test function."""
        display_name = f"[{model}] {name}" if model else name
        def decorator(func):
            def wrapper():
                try:
                    func()
                    self.results.append(TestResult(display_name, True))
                    print(f"  ✅ {display_name}")
                except AssertionError as e:
                    self.results.append(TestResult(display_name, False, str(e)))
                    print(f"  ❌ {display_name}")
                    print(f"     └─ {e}")
                except Exception as e:
                    self.results.append(TestResult(display_name, False, str(e)))
                    print(f"  💥 {display_name}")
                    print(f"     └─ {e}")
            self._tests.append(wrapper)
            return wrapper
        return decorator
    
    def run_all(self):
        """Execute all registered tests."""
        print("═" * 65)
        print("🧪 CLARISSA Voice Module - SPE Benchmark Test Suite")
        print("═" * 65)
        print()
        
        for test_func in self._tests:
            test_func()
        
        passed = sum(1 for r in self.results if r.passed)
        failed = len(self.results) - passed
        
        print()
        print("─" * 65)
        if failed == 0:
            print(f"✅ ALL {passed} TESTS PASSED!")
        else:
            print(f"❌ {failed} FAILED, {passed} passed")
        print("═" * 65)
        
        return failed == 0

runner = SPETestRunner()

# ═══════════════════════════════════════════════════════════════════════════
# GENERAL COMMANDS
# ═══════════════════════════════════════════════════════════════════════════

@runner.test("Cancel/Stop commands")
def test_cancel():
    for cmd in ["cancel", "stop", "abort", "quit"]:
        r = parse_intent(cmd)
        assert r.type == IntentType.CANCEL, f"'{cmd}' → {r.type}"

@runner.test("Confirm commands")
def test_confirm():
    for cmd in ["yes", "yes run it", "confirm", "ok", "do it"]:
        r = parse_intent(cmd)
        assert r.type == IntentType.CONFIRM, f"'{cmd}' → {r.type}"

@runner.test("Help command")
def test_help():
    assert parse_intent("help").type == IntentType.HELP

# ═══════════════════════════════════════════════════════════════════════════
# SPE1 - Simple Black-Oil Model (10x10x3, gas injection)
# ═══════════════════════════════════════════════════════════════════════════

@runner.test("Show pressure distribution", model="SPE1")
def test_spe1_pressure():
    r = parse_intent("show me the pressure distribution")
    assert r.type == IntentType.VISUALIZE_PROPERTY
    assert r.slots.get("property") == "pressure"

@runner.test("Query GOR at producer", model="SPE1")
def test_spe1_gor():
    r = parse_intent("what is the gas oil ratio at the producer")
    assert r.type == IntentType.QUERY_VALUE
    assert "gor" in r.slots.get("property", "").lower() or "gas" in r.slots.get("property", "").lower()

@runner.test("Oil saturation at layer 2", model="SPE1")
def test_spe1_oil_sat():
    r = parse_intent("display the oil saturation at layer 2")
    assert r.type == IntentType.VISUALIZE_PROPERTY
    assert r.slots.get("layer") == 2

@runner.test("3D porosity visualization", model="SPE1")
def test_spe1_poro_3d():
    r = parse_intent("show porosity in 3D")
    assert r.type == IntentType.VISUALIZE_PROPERTY
    assert r.slots.get("property") == "porosity"

# ═══════════════════════════════════════════════════════════════════════════
# SPE9 - Waterflood Model (24x25x15, 26 wells)
# ═══════════════════════════════════════════════════════════════════════════

@runner.test("Water saturation at day 500", model="SPE9")
def test_spe9_wsat():
    r = parse_intent("show water saturation at day 500")
    assert r.type == IntentType.VISUALIZE_PROPERTY
    assert r.slots.get("property") == "water_saturation"
    assert r.slots.get("time_days") == 500

@runner.test("Water cut at producer 10", model="SPE9")
def test_spe9_wct():
    r = parse_intent("what is the water cut at producer 10")
    assert r.type == IntentType.QUERY_VALUE
    assert r.slots.get("property") == "water_cut"

@runner.test("Permeability at layer 8", model="SPE9")
def test_spe9_perm():
    r = parse_intent("display the permeability at layer 8")
    assert r.type == IntentType.VISUALIZE_PROPERTY
    assert r.slots.get("property") == "permeability"
    assert r.slots.get("layer") == 8

@runner.test("Field oil production rate", model="SPE9")
def test_spe9_fopr():
    r = parse_intent("show field oil production rate")
    assert r.type == IntentType.QUERY_VALUE
    assert "oil" in r.slots.get("property", "").lower()

@runner.test("Cumulative oil production", model="SPE9")
def test_spe9_fopt():
    r = parse_intent("what is the cumulative oil production")
    assert r.type == IntentType.QUERY_VALUE

@runner.test("Pressure at layer 15, time 1000", model="SPE9")
def test_spe9_pressure_complex():
    r = parse_intent("visualize pressure at layer 15 time 1000")
    assert r.type == IntentType.VISUALIZE_PROPERTY
    assert r.slots.get("property") == "pressure"
    assert r.slots.get("layer") == 15
    assert r.slots.get("time_days") == 1000

# ═══════════════════════════════════════════════════════════════════════════
# SPE10 - Upscaling Benchmark (1.1M cells)
# ═══════════════════════════════════════════════════════════════════════════

@runner.test("Permeability heterogeneity", model="SPE10")
def test_spe10_perm():
    r = parse_intent("show the permeability heterogeneity")
    assert r.type == IntentType.VISUALIZE_PROPERTY
    assert r.slots.get("property") == "permeability"

@runner.test("Porosity at layer 50", model="SPE10")
def test_spe10_poro():
    r = parse_intent("display porosity at layer 50")
    assert r.type == IntentType.VISUALIZE_PROPERTY
    assert r.slots.get("property") == "porosity"
    assert r.slots.get("layer") == 50

@runner.test("Pore volume query", model="SPE10")
def test_spe10_pv():
    r = parse_intent("what is the pore volume")
    assert r.type == IntentType.QUERY_VALUE

# ═══════════════════════════════════════════════════════════════════════════
# RUN ALL TESTS
# ═══════════════════════════════════════════════════════════════════════════

runner.run_all()

## 📚 Summary

This notebook demonstrated:

1. **Speech-to-Text** - Whisper API transcription with domain vocabulary
2. **Intent Parsing** - LLM-based command understanding
3. **Slot Extraction** - Property, layer, time parameters
4. **Visualization** - 3D and cross-section views triggered by voice

### Next Steps

- **Full Integration**: Connect to OPM Flow simulations
- **Live Microphone**: Browser-based audio capture
- **TTS Response**: Audio feedback with OpenAI TTS
- **Offline Mode**: Local Whisper for air-gapped deployments

### Resources

- [Voice Input Tutorial](../guides/voice-input-tutorial.md)
- [ADR-028: Voice Architecture](../../architecture/adr/ADR-028-voice-input-architecture.md)
- [CLARISSA Source Code](https://gitlab.com/wolfram_laube/blauweiss_llc/clarissa)

---

*Part of CLARISSA - Conversational Language Agent for Reservoir Simulation*